In [1]:
import numpy as np

from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from scipy import stats

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F
torch.manual_seed(1)

import time
from datetime import datetime

from collections import defaultdict
from collections import Counter

from random import randint
import pickle

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('error')

In [2]:
#training sets
with open('train.en') as f:
    train_en = [l.strip() for l in f.readlines()]
with open('train.fr') as f:
    train_fr = [l.strip() for l in f.readlines()]

#validation sets
with open('val.en') as f:
    val_en = [l.strip() for l in f.readlines()]
with open('val.fr') as f:
    val_fr = [l.strip() for l in f.readlines()]

#test sets
with open('test_2017_flickr.en') as f:
    test_en = [l.strip() for l in f.readlines()]
with open('test_2017_flickr.fr') as f:
    test_fr = [l.strip() for l in f.readlines()]

In [22]:
#TODO
# 0 PAD - padding 0 for convenience in masking
# 1 BOS - beginning of sentence
# 2 EOS - end of sentence
# 3 UNK - unknown

In [2]:
max_sentence_length = 50

In [3]:
def tokens_sentences(sentences):
    tokens_list = []
    sentence_list = []
    for s in sentences:
        split_sent = s.split()
        sentence = []
        for w in split_sent:

            tokens_list.append(w)
            sentence.append(w)

        sentence_list.append(sentence)
    
    return tokens_list, sentence_list

In [4]:
tokens_list_en, sentence_list_en = tokens_sentences(train_en)

In [8]:
tokens_train_en = list(sorted(set(tokens_list_en)))
print('Vocabulary size', len(tokens_train_en))

count_tokens_train_en = Counter(tokens_list_en)

Vocabulary size 15456


In [13]:
def get_id_dicts(tokens):
    #default dictionary key:id value:token
    id2tokens = defaultdict(str)

    for i in range(len(tokens)):
        id2tokens[i] = tokens[i]

    #default dictionary key:token value:id
    tokens2id = defaultdict(int)

    for ind in id2tokens:
        tokens2id[id2tokens[ind]] = ind

    return tokens2id, id2tokens

In [14]:
tokens2id_en, id2tokens_en = get_id_dicts(tokens_train_en)

vocabulary_size_train_en = len(tokens2id_en)
print(vocabulary_size_train_en)

15456


In [19]:
def convert_corpus2id(sentence_list, tokens2id):
    
    #convert dataset to ids
    corpus2id = []
    
    for s in sentence_list:
    
        sentence2id = []

        for w in s:
            word_id = tokens2id[w]
            sentence2id.append(word_id)

        corpus2id.append(sentence2id)
    
    return corpus2id

In [20]:
train_en_2id = convert_corpus2id(sentence_list_en, tokens2id_en)

In [21]:
train_en_2id[0]

[2017, 15429, 2125, 8935, 2551, 9950, 9568, 8979, 3793]

In [ ]:
class EncoderDecoderAttention(nn.Module):
    def __init__(self, vocab_size_en,vocab_size_fr, w_embedding_dim, p_embedding_dim, max_sentence_length):
        super(EmbedAlign, self).__init__()
        
        self.vocab_size_en = vocab_size_en
        self.vocab_size_fr = vocab_size_fr
        self.max_sentence_length = max_sentence_length
        self.w_embedding_dim = w_embedding_dim
        self.p_embedding_dim = p_embedding_dim
        
        #encoder
        self.w_embeddings = nn.Embedding(self.vocab_size_en, self.w_embedding_dim)
        self.p_embeddings = nn.Embedding(self.max_sentence_length, self.p_embedding_dim)
        
        #self.attention = 
        
        self.attention_projection = nn.Linear(self.embedding_dim, self.embedding_dim)
        #do we use non-linearity after attention
        
        #decoder   
        self.rnn = nn.RNN(self.embedding_dim, self.embedding_dim)
        
        #self.bidirLSTM = nn.LSTM(self.embedding_dim, self.embedding_dim, bidirectional=True)
        #TODO: LSTM, GRU 
        
        #TODO: DROPOUT
        
        
    def forward(self, batch_en, batch_fr, pos_fr, lens_fr):
        
        
        w_out = self.w_embeddings(batch_fr)
        
        p_out = self.p_embeddings(pos_fr)
        
        e_out = torch.cat((w_out, p_out), 0)
        
        #avg_out = for each sentence
        
        #if EOS for encoder, move on the decoder
        
        attention_matrices = self.attention_projection(e_out)
        
        #a = torch.matmul(attention_matrices, hidden_from_decoder)
        
        #softmax_a = a/a.sum()
        
        #context = e_out*a
     
        
        #a = F.softmax(a, dim=0) 
        
        #input embedding
        #set hidden at the beginning
        #get rnn output
        #apply softmax

        #feed actual word for training
        #feed previous word for testing


#             embeddings = self.w_embeddings(word_x)
#             #view_shape = embeddings.shape[0]
#             output, (hidden, cell) = self.bidirLSTM(embeddings.view(1, 1, -1)) 

#             hid_f = hidden[0]
#             hid_b = hidden[1]

        final_out = 0
            
        return final_out

epochs = 10
learning_rate = 0.01
w_embedding_dim = 100
p_embedding_dim = 100

model = EncoderDecoderAttention(vocabulary_size_en, vocabulary_size_fr, embedding_dim) #pad
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

losses = []
avg_losses = []

portion = 1000

print('epoch, total loss, average loss, duration')
for e in range(epochs):
    
    then = datetime.now()
    
    total_loss = 0.0
    for b in range(portion):

        mu_i = []
        sigma_i = []
        z_i = []
        best_alignments = []
        
        if len(batches_en[b]) > 0 and len(batches_fr[b]) > 0:
            batch_en = torch.tensor(np.asarray(batches_en[b]), dtype= torch.long)

            batch_fr = torch.tensor(np.asarray(batches_fr[b]), dtype= torch.long)

            optimizer.zero_grad()

            loss, mu_i, sigma_i, xi,yi,z_i, best_alignments = model(batch_en, batch_fr,mu_i, sigma_i, z_i, best_alignments)

            loss.backward()

            optimizer.step()

            total_loss += loss.item() 
    
    now = datetime.now()
        
    losses.append(total_loss)
    
    avg_loss = np.mean(losses)/len(batches_en)
    
    print(e, total_loss, avg_loss, now-then)
    
    avg_losses.append(avg_loss)
    

with open('model_embed' + str(portion) + '.pickle','wb') as file:
    pickle.dump(model,file)
      
iteration= list(range(len(avg_losses)))

plt.plot(iteration, avg_losses)
plt.xlabel("Iterations for MT")
plt.ylabel('Average loss')
plt.title('Evolution of the loss as a function of the iteration')
plt.savefig("mt" + str(portion)+".png")
plt.show()